In [1]:
import csv

In [2]:
filename="dataset.csv"

In [3]:
fields=[]
rows=[]
with open(filename,'r') as csvfile:
    csvreader=csv.reader(csvfile)
    fields=next(csvreader)
    for row in csvreader:
        rows.append(row)
    print("Total numer of rows: %d"%(csvreader.line_num))


Total numer of rows: 116


In [4]:
import paho.mqtt.publish as publish

In [5]:
channelID = "1427569"
apiKey = "EHIRJJL2BQAGU6Q5"
topic = "channels/" + channelID + "/publish/" + apiKey
mqttHost = "mqtt.thingspeak.com"
tTransport = "tcp"
tPort = 1883
tTLS = None

In [6]:
class CircularQueue():
 
    # constructor
    def __init__(self, size): # initializing the class
        self.size = size
         
        # initializing queue with none
        self.queue = [None for i in range(size)]
        self.front = self.rear = -1
 
    def enqueue(self, data):
         
        # condition if queue is full
        if ((self.rear + 1) % self.size == self.front):
            print(" Queue is Full\n")
             
        # condition for empty queue
        elif (self.front == -1):
            self.front = 0
            self.rear = 0
            self.queue[self.rear] = data
        else:
             
            # next position of rear
            self.rear = (self.rear + 1) % self.size
            self.queue[self.rear] = data
             
    def dequeue(self):
        if (self.front == -1): # condition for empty queue
            #print ("Queue is Empty\n")
            return []
             
        # condition for only one element
        elif (self.front == self.rear):
            temp=self.queue[self.front]
            self.front = -1
            self.rear = -1
            return temp
        else:
            temp = self.queue[self.front]
            self.front = (self.front + 1) % self.size
            return temp
 
    def display(self):
     
        # condition for empty queue
        if(self.front == -1):
            print ("Queue is Empty")
 
        elif (self.rear >= self.front):
            print("Elements in the circular queue are:",
                                              end = " ")
            for i in range(self.front, self.rear + 1):
                print(self.queue[i], end = " ")
            print ()
 
        else:
            print ("Elements in Circular Queue are:",
                                           end = " ")
            for i in range(self.front, self.size):
                print(self.queue[i], end = " ")
            for i in range(0, self.rear + 1):
                print(self.queue[i], end = " ")
            print ()
 
        if ((self.rear + 1) % self.size == self.front):
            print("Queue is Full")
 


In [7]:
ob = CircularQueue(5)
import time

#Sending Data
def edge():
    for [timestamp,values,Sensor] in rows:
        tPayload = "field1=" + str(timestamp)+ "&field2=" + str(values)+ "&field3=" + str(Sensor[-1])
        try:
           publish.single(topic, payload=tPayload, hostname=mqttHost, port=tPort, tls=tTLS, transport=tTransport)
        

        except (KeyboardInterrupt):
           break
        except:
            print ("[INFO] Failure in sending data")
            ob.enqueue([timestamp,values,Sensor])

        time.sleep(60)
        
            
#Buffered data    
def deq():
    buffer=ob.dequeue()
    while(True):
        
        if len(buffer)!=0:
            tPayload = "field1=" + str(buffer[0])+ "&field2=" + str(buffer[1])+ "&field3=" + str(buffer[2])
            try:
                publish.single(topic, payload=tPayload, hostname=mqttHost, port=tPort, tls=tTLS, transport=tTransport)
                print('Buffered data published')
                buffer=[]
              

            except (KeyboardInterrupt):
                break
            except:
                pass
            time.sleep(5)
        else:
            buffer=ob.dequeue()  

In [ ]:
# importing the threading module
import threading
  
if __name__ == "__main__":
    # creating thread
    t1 = threading.Thread(target=edge, args=())
    t2 = threading.Thread(target=deq, args=())

    # starting thread 1
    t1.start()
    # starting thread 2
    t2.start()

    # wait until thread 1 is completely executed
    t1.join()
    # wait until thread 2 is completely executed
    t2.join() 

    # both threads completely executed
    print("Done!")